In [1]:
import numpy as np

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from skimage.io import imread
from skimage.color import gray2rgb
from skimage.color import rgb2hsv
from skimage.transform import resize
from skimage.viewer import ImageViewer
import os
import skimage
from sklearn.metrics import confusion_matrix

In [4]:
image_dim = 64
imgs = np.zeros((8987,image_dim,image_dim,3))
k=0
y = np.zeros((8987,1))
for l,i in enumerate(os.listdir("Images")):
    print("Reading ",i," ...")
    for j in os.listdir(os.path.join("Images",i)):
        x = imread(os.path.join('Images',i,j))
        if len(x.shape) == 2:
            x = gray2rgb(x)
        imgs[k] = resize(x, (image_dim,image_dim,3)).reshape((1,image_dim,image_dim,3))
        y[k] = l
        k=k+1
        if(k%1000 == 0):
            print(k)

Reading  n00015388  ...
1000
Reading  n00007846  ...
2000
Reading  n00017222  ...
3000
4000
Reading  n00523513  ...
5000
Reading  n12992868  ...
6000
7000
Reading  n09287968  ...
8000


In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test,y_train,y_test= train_test_split(imgs,y, test_size=0.20, random_state=42)

In [6]:
#euclidean predictions
def nn_pred_euclidean(reduced_img_tr,reduced_img_te):
    pred = np.zeros((len(x_test),1)).astype('int')
    for i,v in enumerate(reduced_img_te):
        a = reduced_img_tr - v
        b = np.linalg.norm(a,axis = 1).argsort()[0:5]
        b = list(y_train[b].reshape(5,).astype('int'))
        pred[i] = max(b,key = b.count)
    return pred

In [7]:
#pearson predictions
def nn_pred_pear(reduced_img_tr,reduced_img_te):
    Y_Y = reduced_img_tr - np.mean(reduced_img_tr,axis = 1).reshape(7189,1)
    pred = np.zeros((len(reduced_img_te),1)).astype('int')
    for i,v in enumerate(reduced_img_te):
        X_X = (v - np.mean(v))
        num = np.dot((v - np.mean(v)).reshape(1,reduced_img_te.shape[1]),reduced_img_tr.transpose())
        Y_norm = np.linalg.norm(Y_Y,axis = 1)
        X_norm = np.linalg.norm(X_X)
        den = X_norm*Y_norm
        b = (num/den).reshape(len(reduced_img_tr),).argsort()[-5:][::-1]
        b = list(y_train[b].reshape(5,).astype('int'))
        pred[i] = max(b,key = b.count)
    return pred    

## AUTOENCODER

I have used the architecture which gave the least test mse for last question

In [8]:
#Autoencoders 
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

input_img = Input(shape=(image_dim, image_dim, 3))
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
encoded = Conv2D(32, (3, 3), activation='relu', padding='same')(x)

x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, outputs = decoded)
encoder = Model(input_img, encoded)

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy', metrics = ['mse'])

Using TensorFlow backend.


In [9]:
from keras.callbacks import TensorBoard

autoencoder.fit(x_train, x_train,
                epochs=13,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

Train on 7189 samples, validate on 1798 samples
Epoch 1/13
7189/7189 [==============================] - 179s - loss: 0.6594 - mean_squared_error: 0.0630 - val_loss: 0.6106 - val_mean_squared_error: 0.0392
Epoch 2/13
7189/7189 [==============================] - 191s - loss: 0.6067 - mean_squared_error: 0.0391 - val_loss: 0.5911 - val_mean_squared_error: 0.0307
Epoch 3/13
7189/7189 [==============================] - 185s - loss: 0.5907 - mean_squared_error: 0.0322 - val_loss: 0.5933 - val_mean_squared_error: 0.0315
Epoch 4/13
7189/7189 [==============================] - 181s - loss: 0.5818 - mean_squared_error: 0.0285 - val_loss: 0.5806 - val_mean_squared_error: 0.0265
Epoch 5/13
7189/7189 [==============================] - 203s - loss: 0.5781 - mean_squared_error: 0.0271 - val_loss: 0.5773 - val_mean_squared_error: 0.0249
Epoch 6/13
7189/7189 [==============================] - 177s - loss: 0.5731 - mean_squared_error: 0.0251 - val_loss: 0.5754 - val_mean_squared_error: 0.0245
Epoch 7/13

In [10]:
encoded_imgs_test = encoder.predict(x_test).reshape((len(x_test),np.prod([16,16,32])))
encoded_imgs_train = encoder.predict(x_train).reshape((len(x_train),np.prod([16,16,32])))

In [33]:
pred_AE_eu = nn_pred_euclidean(encoded_imgs_train,encoded_imgs_test)
cf_mat_AE_eu = confusion_matrix(y_test, pred_AE_eu)

print ("Confusion matrix 5-nn predictions using euclidean distance for autoencoders \n",cf_mat_AE_eu)
print("\nAccuracy using euclidean distance for autoencoders = ",
      sum(np.diagonal(cf_mat_AE_eu))/len(x_test)*100,"%")

Confusion matrix 5-nn predictions using euclidean distance for autoencoders 
 [[116  17  47  17  17 104]
 [ 41  53  22  19  14  82]
 [ 50   8  99  11  27  63]
 [ 84  13  40  77  16 158]
 [ 46   7  54   8  64  68]
 [ 29   9  26   8  10 274]]

Accuracy using euclidean distance for autoencoders =  37.9866518354 %


In [34]:
pred_AE_pe = nn_pred_pear(encoded_imgs_train,encoded_imgs_test)
cf_mat_AE_pe = confusion_matrix(y_test, pred_AE_pe)

print ("Confusion matrix 5-nn predictions using pearson correlation for autoencoders", cf_mat_AE_pe)
print("\nAccuracy using pearson correlation for autoencoders = ",
      sum(np.diagonal(cf_mat_AE_pe))/len(x_test)*100,"%")

Confusion matrix 5-nn predictions using pearson correlation for autoencoders [[ 82  39  36  29  21 111]
 [ 26  87  11  20  16  71]
 [ 31  15 102  11  35  64]
 [ 59  24  33 122  19 131]
 [ 35  17  49   6  83  57]
 [ 30  17  17  10   9 273]]

Accuracy using pearson correlation for autoencoders =  41.6573971079 %


## SVD

In [13]:
#SVD
import tensorflow as tf
x_train_svd = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test_svd = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [14]:
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix

svd = TruncatedSVD(n_components=60, n_iter=7, random_state=42)
svd.fit(x_train_svd)

TruncatedSVD(algorithm='randomized', n_components=60, n_iter=7,
       random_state=42, tol=0.0)

**Why 60 Components?**

I tried with different values of eigenvectors i.e. n_components and got best accuracy with 60. Hence I have used n_components = 60

In [15]:
#Variance explaiend by 60 components
print("Variance Explained by 60 components",svd.explained_variance_ratio_.sum())

Variance Explained by 60 components 0.688740006381


In [16]:
train_transformed = svd.transform(x_train_svd)
test_transformed = svd.transform(x_test_svd)

In [29]:
pred_SVD_eu = nn_pred_euclidean(train_transformed,test_transformed)
cf_mat_SVD_eu = confusion_matrix(y_test, pred_SVD_eu)

print ("Confusion matrix 5-nn predictions using euclidean distance for SVD Vectors \n",cf_mat_SVD_eu)
print("\nAccuracy using euclidean distance for RGB Vectors = ",
      sum(np.diagonal(cf_mat_SVD_eu))/len(x_test)*100,"%")

Confusion matrix 5-nn predictions using euclidean distance for SVD Vectors 
 [[ 77  15  73  43  34  76]
 [ 33  43  28  43  30  54]
 [ 36   5 124  18  35  40]
 [ 72   4  47 151  24  90]
 [ 26   5  57  24  95  40]
 [ 24  11  22  36  13 250]]

Accuracy using euclidean distance for RGB Vectors =  41.1568409344 %


In [30]:
pred_SVD_pe = nn_pred_pear(train_transformed,test_transformed)
cf_mat_SVD_pe = confusion_matrix(y_test, pred_SVD_pe)

print ("Confusion matrix 5-nn predictions using euclidean distance for SVD Vectors \n",cf_mat_SVD_pe)
print("\nAccuracy using euclidean distance for RGB Vectors = ",
      sum(np.diagonal(cf_mat_SVD_pe))/len(x_test)*100,"%")

Confusion matrix 5-nn predictions using euclidean distance for SVD Vectors 
 [[ 90  15  51  46  25  91]
 [ 54  34  17  34  23  69]
 [ 29   3 126  17  36  47]
 [ 54  10  33 165  11 115]
 [ 35   6  42  25  80  59]
 [ 28  13  14  29   6 266]]

Accuracy using euclidean distance for RGB Vectors =  42.3248053393 %


## RGB

In [23]:
from collections import Counter
#converting xtrain into rgb
x_train_rgb = np.zeros((len(x_train),256*3))
for idx,i in enumerate(x_train):
    i = skimage.img_as_ubyte(i) #as the values were normalized after resizing converting them back into rgb
    # creating a dictionary to count the frequency of each value from 0-255
    r = Counter(i[:,:,0].reshape(i.shape[0]*i.shape[1])) 
    g = Counter(i[:,:,1].reshape(i.shape[0]*i.shape[1]))
    b = Counter(i[:,:,2].reshape(i.shape[0]*i.shape[1]))
    rgb = np.zeros((3,256))
    #creating rgb vectors seperately
    rgb[0,np.array(list(r.keys()))] = list(r.values())
    rgb[1,np.array(list(g.keys()))] = list(g.values())
    rgb[2,np.array(list(b.keys()))] = list(b.values())
    rgb = rgb.reshape(256*3)
    x_train_rgb[idx] = rgb
    
x_test_rgb = np.zeros((len(x_test),256*3))
for idx,i in enumerate(x_test):
    i = skimage.img_as_ubyte(i) #as the values were normalized after resizing converting them back into rgb
    # creating a dictionary to count the frequency of each value from 0-255
    r = Counter(i[:,:,0].reshape(64*64)) 
    g = Counter(i[:,:,1].reshape(64*64))
    b = Counter(i[:,:,2].reshape(64*64))
    rgb = np.zeros((3,256))
    #creating rgb vectors seperately
    rgb[0,np.array(list(r.keys()))] = list(r.values())
    rgb[1,np.array(list(g.keys()))] = list(g.values())
    rgb[2,np.array(list(b.keys()))] = list(b.values())
    rgb = rgb.reshape(256*3)
    x_test_rgb[idx] = rgb

In [24]:
pred_RGB_eu = nn_pred_euclidean(x_train_rgb,x_test_rgb)
cf_mat_RGB_eu = confusion_matrix(y_test, pred_RGB_eu)

print ("Confusion matrix 5-nn predictions using euclidean distance for RGB Vectors \n",cf_mat_RGB_eu)
print("\nAccuracy using euclidean distance for RGB Vectors = ",
      sum(np.diagonal(cf_mat_RGB_eu))/len(x_test)*100,"%")

Confusion matrix 5-nn predictions using euclidean distance for RGB Vectors 
 [[ 84  21  44  26  87  56]
 [ 26  58  13  26  77  31]
 [ 25   9 118  10  66  30]
 [ 73  33  37 113  88  44]
 [ 28  13  29   7 150  20]
 [ 61  24  34  28  59 150]]

Accuracy using euclidean distance for RGB Vectors =  37.4304783092 %


In [25]:
pred_RGB_pe = nn_pred_pear(x_train_rgb,x_test_rgb)
cf_mat_RGB_pe = confusion_matrix(y_test, pred_RGB_pe)

print ("Confusion matrix 5-nn predictions using euclidean distance for RGB Vectors\n",cf_mat_RGB_pe)
print("\nAccuracy using euclidean distance for RGB Vectors = ",
      sum(np.diagonal(cf_mat_RGB_pe))/len(x_test)*100,"%")

Confusion matrix 5-nn predictions using euclidean distance for RGB Vectors
 [[ 85  29  49  28  53  74]
 [ 32  76  11  19  49  44]
 [ 35  10 112  13  51  37]
 [ 69  32  50 126  47  64]
 [ 37  17  36   4 117  36]
 [ 55  28  33  24  28 188]]

Accuracy using euclidean distance for RGB Vectors =  39.1546162403 %


## HSV

In [26]:
#converting xtrain into rgb
x_train_hsv = np.zeros((len(x_train),(256*2+180)))
for idx,i in enumerate(x_train):
    h, _ = np.histogram(i[:,:,0], bins = 180 )
    s, _ = np.histogram(i[:,:,1], bins = 256 )
    v, _ = np.histogram(i[:,:,2], bins = 256 )
    x_train_hsv[idx] = np.concatenate((h,s,v), axis=0)
    
x_test_hsv = np.zeros((len(x_test),(256*2+180)))
for idx,i in enumerate(x_test):
    h, _ = np.histogram(i[:,:,0], bins = 180 )
    s, _ = np.histogram(i[:,:,1], bins = 256 )
    v, _ = np.histogram(i[:,:,2], bins = 256 )
    x_test_hsv[idx] = np.concatenate((h,s,v), axis=0)

Prediction for HSV Histogram

In [31]:
pred_HSV_eu = nn_pred_euclidean(x_train_hsv,x_test_hsv)
cf_mat_HSV_eu = confusion_matrix(y_test, pred_HSV_eu)

print ("Confusion matrix 5-nn predictions using euclidean distance for HSV Vectors \n",cf_mat_HSV_eu)
print("\nAccuracy using euclidean distance for RGB Vectors = ",
      sum(np.diagonal(cf_mat_HSV_eu))/len(x_test)*100,"%")

Confusion matrix 5-nn predictions using euclidean distance for HSV Vectors 
 [[ 82  26  47  18  79  66]
 [ 40  57  14  22  67  31]
 [ 37   7 102  11  61  40]
 [ 77  29  25 115  93  49]
 [ 35  13  35   6 138  20]
 [ 60  26  25  23  51 171]]

Accuracy using euclidean distance for RGB Vectors =  36.9855394883 %


In [32]:
pred_HSV_pe = nn_pred_pear(x_train_hsv,x_test_hsv)
cf_mat_HSV_pe = confusion_matrix(y_test, pred_HSV_pe)

print ("Confusion matrix 5-nn predictions using euclidean distance for HSV Vectors \n",cf_mat_HSV_pe)
print("\nAccuracy using euclidean distance for RGB Vectors = ",
      sum(np.diagonal(cf_mat_HSV_pe))/len(x_test)*100,"%")

Confusion matrix 5-nn predictions using euclidean distance for HSV Vectors 
 [[ 76  21  56  32  57  76]
 [ 39  72  14  17  49  40]
 [ 35  13 105  21  48  36]
 [ 80  32  34 139  50  53]
 [ 26  21  37   6 122  35]
 [ 54  24  27  19  31 201]]

Accuracy using euclidean distance for RGB Vectors =  39.766407119 %


### Comparison of Accuracy for different methods

                                 
                  
1] Autoencoder  :  Euclidean  37.98% , Pearson      41.66%

2] SVD          :  Euclidean  41.16% , Pearson      42.32%

3] RGB Histogram:  Euclidean  37.43% , Pearson      39.15%

4] HSV Histogram:  Euclidean  36.96% , Pearson 39.77%

The highest accuracy for test data was obtained for SVD vectors. Train and test data used for each of the method is same